In [1]:
import os 

In [2]:
%pwd

'd:\\Chicken_Disease_Classification\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\Chicken_Disease_Classification'

In [19]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
     root_dir: Path 
     source_URL: Path 
     local_data_file: Path 
     unzip_dir: Path 

In [20]:
from src.CnnClassifier.constants import *
from src.CnnClassifier.utils.common import read_yaml,create_directories
# from box import BoxException


In [25]:
class ConfigurationManager:
     def __init__(
          self,
          config_filepath=CONFIG_FILE_PATH,
          params_filepath=PARAMS_FILE_PATH
          
     ):
          self.config=read_yaml(config_filepath)
          self.params=read_yaml(params_filepath)
          create_directories([self.config.artifacts_root])
     
     def get_data_ingestion_config(self)->DataIngestionConfig:
          config=self.config.data_ingestion
          create_directories([config.root_dir])
          data_ingestion_config=DataIngestionConfig(
               root_dir=config.root_dir,
               source_URL=config.source_URL,
               local_data_file=config.local_data_file,
               unzip_dir=config.unzip_dir,
          )
          return data_ingestion_config
          

In [26]:
import os 
import urllib.request as request
import zipfile
from src.CnnClassifier.logger.logg import logging
from src.CnnClassifier.utils.common import get_size

In [29]:
class DataIngestion:
     def __init__(self,config:DataIngestionConfig):
          self.config=config
          
     def download_file(self):
          if not os.path.exists(self.config.local_data_file):
               filename,headers=request.urlretrieve(
                    url=self.config.source_URL,
                    filename=self.config.local_data_file
               )
               file_path = Path(self.config.local_data_file)
               logging.info(f"Downloaded {get_size(file_path)} from {self.config.source_URL}")
          else:
               logging.info(f"{self.config.local_data_file} already exists")
               
     def extract_zip_file(self):
          unzip_path=self.config.unzip_dir
          os.makedirs(unzip_path,exist_ok=True)
          with zipfile.ZipFile(self.config.local_data_file,"r") as zip_ref:
               zip_ref.extractall(unzip_path)
               logging.info(f"Extracted {self.config.local_data_file} to {self.config.unzip_dir}")

In [30]:
try:
     config=ConfigurationManager()
     data_ingestion_config=config.get_data_ingestion_config()
     dataingestion=DataIngestion(data_ingestion_config)
     dataingestion.download_file()
     dataingestion.extract_zip_file()
except Exception as e:
     raise(e)